# Calculando SVF para um ponto com Pandas

Neste experimento vamos calcular para um ponto definido o SVF utilizando Pandas. Para isso será utilizado uma técnica de calcular para cada ponto o $\theta$ e o $\phi$ a partir do ponto escolhido e então agrupar pelo particionamento da semiesfera definida

In [13]:
import pandas as pd
import geopandas as gpd
import pdal
import numpy as np

In [14]:
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime


In [15]:
pipeline="""{
  "pipeline": [
    {
        "type": "readers.las",
        "filename": "arquivos/MDS/MDS_3314-231.laz"
    },
    {
        "type": "filters.sample",
        "radius": 1
    }
  ]
}"""

# pipeline="""{
#   "pipeline": [
#     {
#         "type": "readers.las",
#         "filename": "arquivos/MDS/MDS_3314-231.laz"
#     }
#   ]
# }"""

In [16]:
gdf_observ = gpd.read_file('arquivos/pontos_validacao/pontos_preliminares.gpkg')

In [17]:
# gdf_observ = gdf_observ.to_crs(31983)

In [18]:
gdf_observ

,SCM,geometry
0,3314-231,POINT Z (333344.322 7395282.674 724.891)
1,3331-132,POINT Z (324531.390 7385905.216 757.406)
2,4431-222,POINT Z (352135.100 7386243.508 789.996)
3,3314-231,POINT Z (333349.447 7395315.760 724.728)
4,3314-231,POINT Z (333353.608 7395345.881 724.915)
5,3314-231,POINT Z (333367.084 7395363.517 724.698)
6,3314-231,POINT Z (333354.996 7395284.648 724.802)
7,3314-231,POINT Z (333357.175 7395273.947 724.968)
8,3314-231,POINT Z (333347.862 7395272.560 724.917)
9,3314-231,POINT Z (333361.337 7395326.659 724.808)


In [19]:
[list(x.coords)[0][2] for x in gdf_observ.geometry]

[724.8905029296875,
 757.4063110351562,
 789.9957275390625,
 724.7280883789062,
 724.9146118164062,
 724.698486328125,
 724.8023071289062,
 724.9680786132812,
 724.916748046875,
 724.8078002929688,
 724.7515258789062,
 724.8031005859375,
 724.9634399414062,
 724.8020629882812,
 724.7892456054688,
 724.4847412109375,
 732.81640625,
 733.6478271484375,
 732.2969360351562,
 724.7593383789062,
 737.9701538085938,
 724.9641723632812,
 724.968017578125,
 725.2062377929688]

In [20]:
pt_obs = gdf_observ[gdf_observ.SCM == '3314-231'].geometry#.coords[:]

In [21]:
[list(x.coords)[0] for x in pt_obs.geometry]

[(333344.32172328787, 7395282.674381973, 724.8905029296875),
 (333349.4469259395, 7395315.759854813, 724.7280883789062),
 (333353.60836743616, 7395345.880764694, 724.9146118164062),
 (333367.0835113299, 7395363.517350084, 724.698486328125),
 (333354.99551460164, 7395284.648125529, 724.8023071289062),
 (333357.17531729036, 7395273.947275966, 724.9680786132812),
 (333347.8616148932, 7395272.5601288015, 724.916748046875),
 (333361.336758787, 7395326.658868257, 724.8078002929688),
 (333399.3842238988, 7395358.365089184, 724.7515258789062),
 (333403.5456653954, 7395323.488246164, 724.8031005859375),
 (333409.68874569994, 7395362.52653068, 724.9634399414062),
 (333398.19524061406, 7395286.629764337, 724.8020629882812),
 (333388.6833743361, 7395285.837108814, 724.7892456054688),
 (333395.420946283, 7395324.082737806, 724.4847412109375),
 (333297.13166141073, 7395248.582299229, 732.81640625),
 (333278.7519614674, 7395273.204161419, 733.6478271484375),
 (333315.5113613541, 7395366.489808304, 73

In [22]:
# pt_obs

## Carregando dados LiDAR 3D

In [23]:
r = pdal.Pipeline(pipeline)
r.validate()
r.execute()
arrays = r.arrays

In [24]:
df = pd.DataFrame(arrays[0])

In [25]:
df_mds = \
df[(df.Classification == 2) | (df.Classification == 6)]\
                          [['X', 'Y', 'Z']]#\
# .astype({'X': 'int32', \
#         'Y': 'int32', \
#         'Z': 'int16'})

In [26]:
df_mdt = \
df[df.Classification == 2][['X', 'Y', 'Z']]

In [27]:
df

,X,Y,Z,Intensity,ReturnNumber,NumberOfReturns,ScanDirectionFlag,EdgeOfFlightLine,Classification,ScanAngleRank,UserData,PointSourceId,GpsTime
0,333499.29,7395564.35,735.34,18,1,1,0,0,6,-14.0,17,6,356569.366065
1,333499.23,7395565.79,735.41,18,1,1,0,0,6,-14.0,17,6,356569.366090
2,333499.20,7395566.79,735.25,15,1,1,0,0,6,-14.0,17,6,356569.366105
3,333499.15,7395568.21,735.37,14,1,1,0,0,6,-14.0,17,6,356569.366129
4,333499.10,7395569.68,735.30,17,1,1,0,0,6,-15.0,17,6,356569.366153
...,...,...,...,...,...,...,...,...,...,...,...,...,...
471957,333523.36,7395570.78,735.52,2,1,2,1,0,19,13.0,40,6,363044.324524
471958,333521.03,7395570.91,731.07,5,1,1,1,0,6,13.0,40,6,363044.342732
471959,333520.96,7395571.00,732.50,8,1,1,1,0,6,13.0,40,6,363044.342740
471960,333521.09,7395571.84,731.83,18,1,1,1,0,6,13.0,40,6,363044.360909


In [28]:
# distance = np.sqrt((df_mdt.X - pt_obs[0])**2 + (df_mdt.Y - pt_obs[1])**2)

In [29]:
# distance.sort_values()[:3].index

In [30]:
# df_mdt

## Dividindo o domo celeste

In [31]:
fusos = 60
meridianos = 30

In [32]:
thetas, delta_theta = np.linspace(-np.pi, np.pi, fusos, retstep=True, endpoint=False)
thetas

array([-3.14159265, -3.0368729 , -2.93215314, -2.82743339, -2.72271363,
       -2.61799388, -2.51327412, -2.40855437, -2.30383461, -2.19911486,
       -2.0943951 , -1.98967535, -1.88495559, -1.78023584, -1.67551608,
       -1.57079633, -1.46607657, -1.36135682, -1.25663706, -1.15191731,
       -1.04719755, -0.9424778 , -0.83775804, -0.73303829, -0.62831853,
       -0.52359878, -0.41887902, -0.31415927, -0.20943951, -0.10471976,
        0.        ,  0.10471976,  0.20943951,  0.31415927,  0.41887902,
        0.52359878,  0.62831853,  0.73303829,  0.83775804,  0.9424778 ,
        1.04719755,  1.15191731,  1.25663706,  1.36135682,  1.46607657,
        1.57079633,  1.67551608,  1.78023584,  1.88495559,  1.98967535,
        2.0943951 ,  2.19911486,  2.30383461,  2.40855437,  2.51327412,
        2.61799388,  2.72271363,  2.82743339,  2.93215314,  3.0368729 ])

In [33]:
phis = np.linspace(1., 0., meridianos, retstep=True, endpoint=False)
phis

(array([1.        , 0.96666667, 0.93333333, 0.9       , 0.86666667,
        0.83333333, 0.8       , 0.76666667, 0.73333333, 0.7       ,
        0.66666667, 0.63333333, 0.6       , 0.56666667, 0.53333333,
        0.5       , 0.46666667, 0.43333333, 0.4       , 0.36666667,
        0.33333333, 0.3       , 0.26666667, 0.23333333, 0.2       ,
        0.16666667, 0.13333333, 0.1       , 0.06666667, 0.03333333]),
 -0.03333333333333333)

In [34]:
np.arccos(phis[0])

array([0.        , 0.25892154, 0.36720802, 0.45102681, 0.52231482,
       0.58568554, 0.64350111, 0.69716313, 0.74758435, 0.79539883,
       0.84106867, 0.88494336, 0.92729522, 0.96834169, 1.00826008,
       1.04719755, 1.0852782 , 1.12260819, 1.15927948, 1.19537272,
       1.23095942, 1.26610367, 1.30086353, 1.33529209, 1.36943841,
       1.40334825, 1.43706474, 1.47062891, 1.50408018, 1.53745682])

## Discretizando $\theta$ e $\phi$

In [35]:
## Dado um angulo theta t em radianos qual seu indice em 
## uma divisão de fusos a cada delta_theta

t = np.pi - 0.000000001
t = np.arctan2(0,-1)
np.floor((t + np.pi)/delta_theta).astype('int')

60

In [36]:
## Dado um angulo phi f em radianos qual seu indice em
## uma divisão de meridianos

f = np.pi/2
indice = np.floor(meridianos - (np.cos(f) / (1 / meridianos))).astype('int')
indice

29

## Calculando $\theta$ e $\phi$ para todos os pontos

Método por contagem de numero de grupos de $\theta$ e $\phi$

In [37]:
def svf_observ(x):
    coords = x.coords
    
    x_mds = df_mds.loc[df_mds.Z >= coords[0][2]].reset_index()
    
    x_mds['theta_idx'] = \
    np.floor(np.arctan2(\
              x_mds.X - coords[0][0],
              x_mds.Y - coords[0][1])/delta_theta).astype('int')
    
    x_mds['phi_idx'] = \
    np.floor(meridianos - (np.cos(\
    np.arctan2(\
               np.sqrt((x_mds.X - coords[0][0])**2 + \
                       (x_mds.Y - coords[0][1])**2), \
               x_mds.Z - coords[0][2])) / \
                           (1 / meridianos))).astype('int')

    
    svf = 1. - (x_mds.groupby(['theta_idx', 'phi_idx'])\
                .ngroups / (fusos * meridianos))
    
    return svf

In [38]:
pt_obs.apply(lambda x: svf_observ(x))

0     0.200000
3     0.256667
4     0.316667
5     0.316667
6     0.207222
7     0.215000
8     0.206111
9     0.253333
10    0.212222
11    0.167222
12    0.226667
13    0.203333
14    0.191111
15    0.161111
16    0.124444
17    0.109444
18    0.145556
19    0.315556
20    0.139444
21    0.438333
22    0.419444
23    0.507222
Name: geometry, dtype: float64

### Calculando o polígono de visibilidade

In [51]:
def svf_polygon(x):
    coords = x.coords
    x_mds = df_mds.loc[df_mds.Z >= coords[0][2]].reset_index()
    
    x_mds['theta'] = \
    thetas[\
    (np.floor(np.arctan2(\
              x_mds.X - coords[0][0],
              x_mds.Y - coords[0][1])/delta_theta) + (fusos / 2)).astype('int')]
    
    x_mds['phi'] = \
    np.floor(meridianos - (np.cos(\
    np.arctan2(\
               np.sqrt((x_mds.X - coords[0][0])**2 + \
                       (x_mds.Y - coords[0][1])**2), \
               x_mds.Z - coords[0][2])) / \
                           (1 / meridianos))).astype('int')
    
    x_mds['distance'] = \
    np.sqrt((x_mds.X - coords[0][0])**2 + \
            (x_mds.Y - coords[0][1])**2)
    
    return x_mds.sort_values(by=['theta'])[['theta', 'X', 'Y']]\
    .groupby(['theta']).min('phi')
    
#     return x_mds

In [52]:
svf_polygon(pt_obs.iloc[0])#\
# .sort_values(by=['theta'])[['theta', 'X', 'Y']]\
# .groupby(['theta']).min('phi')

,X,Y
theta,,
-3.141593,333313.76,7394985.29
-3.036873,333281.68,7394985.28
-2.932153,333248.10,7394985.28
-2.827433,333212.73,7394985.28
-2.722714,333173.52,7394985.30
-2.617994,333128.69,7394985.29
-2.513274,333076.87,7394985.28
-2.408554,333058.67,7394985.35
-2.303835,333058.67,7395026.36


In [ ]:
len(pt_obs)